In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import os
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')

In [3]:
input_state = 2
target = data['theta23']
if input_state == 0:
    data_all = data['ve_dune']
elif input_state == 1:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif input_state == 2:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])

x_train = data_all[:1000]
y_train = target[:1000]
x_train2 = data_all[1000:900000]
y_train2 = target[1000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [4]:
generate = 10
x_train = np.reshape(np.random.poisson(x_train, size = (generate, len(x_train), 244)), (generate*len(x_train), 244))
y_train = np.repeat(y_train, generate)

In [ ]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 21 Complete [00h 01m 30s]
val_loss: 15.230931282043457

Best val_loss So Far: 14.753811836242676
Total elapsed time: 00h 27m 22s

Search: Running Trial #22

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |True              
structured_data...|True              |True              
structured_data...|3                 |3                 
structured_data...|32                |32                
structured_data...|0                 |0                 
structured_data...|1024              |32                
regression_head...|0                 |0                 
optimizer         |adam              |adam              
learning_rate     |0.001             |0.001             
structured_data...|32                |32                

Epoch 1/20
142/142 [==============================] - 4s 25ms/step - loss: 1869.1653 - mean_squared_error: 1869.1653 - val_loss: 1863.1176 - val_mean_squared_error: 1863.1176
Epoch 2/20
142/142 [==============

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 1/20
157/157 [==============================] - 1s 8ms/step - loss: 1945.9170 - mean_squared_error: 1945.9170
Epoch 2/20
157/157 [==============================] - 1s 8ms/step - loss: 1593.5920 - mean_squared_error: 1593.5920
Epoch 3/20
157/157 [==============================] - 1s 8ms/step - loss: 1127.6869 - mean_squared_error: 1127.6869
Epoch 4/20
157/157 [==============================] - 1s 8ms/step - loss: 660.5319 - mean_squared_error: 660.5319
Epoch 5/20
157/157 [==============================] - 1s 8ms/step - loss: 302.8840 - mean_squared_error: 302.8840
Epoch 6/20
157/157 [==============================] - 1s 8ms/step - loss: 107.4898 - mean_squared_error: 107.4898
Epoch 7/20
157/157 [==============================] - 1s 8ms/step - loss: 35.2368 - mean_squared_error: 35.2368
Epoch 8/20
157/157 [==============================] - 1s 8ms/step - loss: 17.3579 - mean_squared_error: 17.3579
Epoch 9/20
157/157 [==============================] - 1s 8ms/step - loss: 13.8105 - me

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

In [ ]:
model.evaluate(x_test, y_test)
pre_test = model.predict(x_test)

In [ ]:
plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
plt.hist(y_test, histtype = 'step', bins = 100, label = 'target')
plt.xlabel('theta23')
plt.ylabel('number')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(pre_test, y_test, '.', alpha = 0.05)
plt.xlabel('prediction')
plt.ylabel('target')
plt.title('theta23')

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/Regression/models_all/0804_theta23_1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)